### **Imports**

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,LayerNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model
import os
from os import walk
import numpy as np
import matplotlib.pyplot as plt

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **DATA**

####**Helper Methods**

In [ ]:
def dir_check(path):
    Path(path).mkdir(parents=True, exist_ok=True)

def path_leaf(path):
    head, tail = ntpath.split(path)

    # if name ends with \\ then tail won't return anything
    # return tail or ntpath.basename(head)

    # since you dont want to return in case its not file
    return tail


def getF(directory, x="both"):
    # x could be "files" or "folders" or "both"

    if directory[-1] != "/":
      directory += "/"

    folders = []
    for f in walk(directory):
      folders.extend(f)
      # ???

    if x == "folders":
      return folders[1]

      
      
        
    if x == "files":
        return folders[2]
    else:
      return folders[1], folders[2]



def show_imgs(path2imgs, percentage2show, imgs2show_per_row):
  if percentage2show > 100 or percentage2show <= 0:
    print("Invalid second argument!!\nPercentage should be 1-100")
    return

  if path2imgs[-1] != "/":
      path2imgs += "/"

  imgs = get_shuffled_file_names(path2imgs, percentage2show)
  print("Images: "+str(len(imgs)))
  # # ???
  # #figure size in inches optional

  
  rcParams['figure.figsize'] = 11 ,8
  imgs_left = len(imgs)

  

  # upr bound is fit
  # jump fit
  # print("bound: "+ str(math.ceil(len(imgs)/imgs2show_per_row)+imgs2show_per_row))
  print("Total Images: "+str(len(imgs)))
  print("bound: " + str(len(imgs)))
  for i in range(0, len(imgs), imgs2show_per_row):
    # print("outer_loop: "+str(i))
    
    if imgs_left >= imgs2show_per_row:
      # 1 row and imgs2show_per_row cols
      figure, axes = plt.subplots(1, imgs2show_per_row)

    else:
      figure, axes = plt.subplots(1, imgs_left)
      imgs2show_per_row = imgs_left
   
   
    for j in range(imgs2show_per_row):
        
        axes[j].imshow( mpimg.imread( path2imgs +  str(imgs[i+j]), 0 ) );
        axes[j].set_title(str(imgs[i+j]))
    
    imgs_left -= imgs2show_per_row
    print("imgs left: "+str(imgs_left))
    


# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

####**GET DATA**

In [ ]:
import os

# zip file download link 
url = "https://storage.googleapis.com/kaggle-data-sets/111880%2F269359%2Fbundle%2Farchive.zip?\
GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1594390547&Signature\
=VE8DnacBU0pZ61PuqpCZieSFPqZR0y%2FsI%2BzIRFYO8cQdKRcvfoeak9i%2BTAHteJcR6jW%2BQGF36GR%2BgAupiMVeg\
mhPvk89HAURvh3gJHT2C5FSI9SmD%2FbzB4CdaXRY1BXeKVF3shVFrD7dVbSl5JD4fNnYyrAWvP%2BK3fApNvAgsLi%2FaoV\
5bo8qA8WpXG6iSIqN5KP4muBvXIoOGhx5gAv7GSkzF8F9JZlRb%2FXZYoJiN3gl%2FMHCWRp73OiRp%2BCMrVRqMJpngebF\
QnSfCy5MnluwE3CI%2FIrJZ6FYbMwSHreqAOnvfp%2FoMzooldMzQIrg9CZUgAF6w5YwGdyOpIs7LtIO8A%3D%3D"

download_path = "/content/download/"
os.makedirs(download_path)


# Download
!wget "$url" -P "$download_path"


# filename
old_filename = getF(download_path,"files")
old_filename_path = download_path + old_filename[0]

new_filename = "a.zip"
new_filename_path = download_path + new_filename

# rename file
!mv "$old_filename_path" "$new_filename_path"


# unzip
path2unzip = "/content/data/"

# os.makedirs(path2unzip)
!unzip "$new_filename_path" -d "$path2unzip"

###**Model**

In [ ]:
# Train Path
train_dir = "/content/data/seg_train/seg_train/"

# Val Path
val_dir = "/content/data/seg_test/seg_test/"

In [ ]:
batch_size = 128
epochs = 3000
IMG_HEIGHT = 227
IMG_WIDTH = 227
IMG_CHANNEL = 3
total_classes = 6

####**Datagenerator and callback**

In [ ]:
# Generator Declaration
train_image_generator = ImageDataGenerator(
                                              rescale=1./255,
                                              rotation_range=45,
                                              width_shift_range=.15,
                                              height_shift_range=.15,
                                              horizontal_flip=True,
                                              zoom_range=0.5
                                        ) # Generator for our training data

validation_image_generator = ImageDataGenerator(
                                         rescale=1./255
                                        ) # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
                                                           batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           class_mode='categorical',
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH)
                                                         )



val_data_gen = validation_image_generator.flow_from_directory(  
                                                              batch_size=batch_size,
                                                              directory=val_dir,
                                                              class_mode='categorical',
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH)
                                                            )

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
checkpoint_filepath = '/content/drive/My Drive/weights/Alex/ensemb/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

####**AlexNet Implementation**

In [ ]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH ,3))
conv1 = Conv2D(96, 11, strides=(4,4) , padding='valid', activation='relu')(inputs)
pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None)(conv1)
a = tf.keras.layers.Lambda(tf.nn.local_response_normalization)
lrn1 = a(pool1)
conv2 = Conv2D(256, 5, padding='same', strides=(1,1) , activation='relu')(lrn1)
pool2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None)(conv2)
b = tf.keras.layers.Lambda(tf.nn.local_response_normalization)
lrn2 = b(pool2)
conv3 = Conv2D(384, 3, padding='same', strides=(1,1), activation='relu')(lrn2)
conv4 = Conv2D(384, 3, padding='same', strides=(1,1), activation='relu')(conv3)
conv5 = Conv2D(256, 3, padding='same', strides=(1,1), activation='relu')(conv4)
conv6 = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None)(conv5)
flat1 = Flatten()(conv6)
dense1 = Dense(4096, activation='relu')(flat1)
drop1 = Dropout(0.5)(dense1)
dense2 = Dense(4096, activation='relu')(drop1)
drop2 = Dropout(0.5)(dense2)
dense3 = Dense(total_classes,activation='softmax')(drop2)

In [ ]:
model = Model(inputs=inputs, outputs=dense3, name="alex_intel_night")

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, "model.png", show_shapes=True)

In [ ]:
opt = tf.keras.optimizers.SGD(
    learning_rate=0.1, momentum=0.1, nesterov=False, name='SGD'
)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=epochs, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              )


####**Transfer Learning (Efficient Net)**

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.InceptionResNetV2(input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNEL),
                                              include_top=False, 
                                              weights='imagenet')

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
base_model.summary()

**Freeze the layers**

In [ ]:
base_model.trainable = False

####**Load pretrained weights of your implemented AlexNet**

In [ ]:
model.load_weights('./checkpoints/cp.ckpt')

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(val_data_gen, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
24/24 [==============================] - 6s 241ms/step - loss: 0.3751 - accuracy: 0.8697
test loss, test acc: [0.37507104873657227, 0.8696666955947876]


In [ ]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
model.summary()

Model: "alex_intel_night"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
lambda (Lambda)              (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_204 (Conv2D)          (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 13, 13, 256)  

In [ ]:
# Better to ckeck types of Both models which are about to make ensemble
# type(base_model)
# type(model)

tensorflow.python.keras.engine.training.Model

####**Ensemble**

In [ ]:
new_model = tf.keras.Sequential([
  base_model,
  # tf.keras.layers.Conv2D(32, 3, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  # tf.keras.layers.GlobalAveragePooling2D(),
  # tf.keras.layers.Dense(5, activation='softmax')


  # Flatten(),
  # Dense(4096, activation='relu'),
  # Dropout(0.5),
  # Dropout(0.5),
  # Dense(total_classes,activation='softmax')

  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])


#####**Load Ensemble checkpoint if any**

In [ ]:
new_model.load_weights(checkpoint_filepath)

###**Explore Ensemble Model**

In [ ]:
new_model.summary()

In [ ]:
# If you want to transfer weights some specific layers of your pretrained
#  AlexNet to specific layers of new_mode better to check shapes of both
new_model.layers[0].output_shape

#####**Evaluate Ensemble trainig**

**First Just see if only training newly added last layers works**

In [ ]:
# opt2 = tf.keras.optimizers.SGD(
#     learning_rate=0.1, momentum=0.1, nesterov=False, name='SGD'
# )
# new_model.compile(loss='categorical_crossentropy', optimizer=opt2, metrics=['accuracy'])

new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = new_model.evaluate(val_data_gen, batch_size=batch_size)
print("test loss, test acc:", results)

#####**Fine Tuning/Further training**

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  780


In [ ]:
# opt2 = tf.keras.optimizers.SGD(
#     learning_rate=0.1, momentum=0.1, nesterov=False, name='SGD'
# )
# new_model.compile(loss='categorical_crossentropy', optimizer=opt2, metrics=['accuracy'])


new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=10, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              
                              )

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=10, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              
                              )

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=30, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              
                              )

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=30, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              
                              )

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=30, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              
                              )

**PLOT**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

**Unfreeze few layers and train**

In [ ]:
base_model.trainable = True

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 150

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
opt2 = tf.keras.optimizers.SGD(
    learning_rate=0.1, momentum=0.1, nesterov=False, name='SGD'
)
new_model.compile(loss='categorical_crossentropy', optimizer=opt2, metrics=['accuracy'])

In [ ]:
history = new_model.fit(
                              train_data_gen,
                              steps_per_epoch=14034//batch_size,
                              epochs=20, validation_data=val_data_gen,
                              validation_steps=3000//batch_size,
                              callbacks=[model_checkpoint_callback]
                              )

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = new_model.evaluate(val_data_gen, batch_size=batch_size)
print("test loss, test acc:", results)

Evaluate on test data
24/24 [==============================] - 35s 1s/step - loss: 0.3048 - accuracy: 0.8917
test loss, test acc: [0.3048091530799866, 0.8916666507720947]


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.save_weights('/content/drive/My Drive/tuk/Alex/intel2/my_model_weights.h5')
model.save('/content/drive/My Drive/tuk/Alex/intel2/my_model.h5')